In [79]:
import pandas as pd
import numpy as np
import math

In [15]:
# sample

data = {'x1': [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3],
        'x2': ['S', 'M', 'M', 'S', 'S', 'S', 'M', 'M', 'L', 'L', 'L', 'M', 'M', 'L', 'L'],
        'y':  [-1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1]
        }
df = pd.DataFrame(data)
print df

    x1 x2  y
0    1  S -1
1    1  M -1
2    1  M  1
3    1  S  1
4    1  S -1
5    2  S -1
6    2  M -1
7    2  M  1
8    2  L  1
9    2  L  1
10   3  L  1
11   3  M  1
12   3  M  1
13   3  L  1
14   3  L -1


In [76]:
# 训练

feature_list = ['x1', 'x2']
#x_feature_df
#x_feature_set = set(data['x1']) # x1 特征取值
#x2_feature_set = set(data['x2']) # x2 特征取值
class_list = list(set(data['y'])) # label 特征取值

#prior_cnt = np.zeros(len(class_set)) # 用来记录每个分类的次数，对应记录的是每个分类的先验概率
#但由于先验概率=每个分类的频次/总样本数，而贝叶斯的分类过程，是比较先验概率*独立条件概率的大小，因此先验概率的分母，也可以忽略不计
#conditional_probability = np.zeros((len(class_set), 2, 1)) # λ=1 拉普拉斯平滑

# 计算各分类和条件的出现频次
class_cnt = df.groupby(['y']).size().to_frame().to_dict('index')
x_feature_cnt = {}
y_x_feature_cnt = {}
for x in feature_list:
    x_feature_cnt[x] = df.groupby([x]).size().to_frame().to_dict('index')
    y_x_feature_cnt[x] = df.groupby(['y', x]).size().to_frame().to_dict('index')
##print class_cnt, x_feature_cnt, y_x_feature_cnt

# 计算条件概率
for x in y_x_feature_cnt:
    for y, x_value in y_x_feature_cnt[x]: 
        # 拉普拉斯平滑
        #print y, x_value, y_x_feature_cnt[x][(y, x_value)][0], class_cnt[y][0], len(x_feature_cnt[x])
        y_x_feature_cnt[x][(y, x_value)] = ( # 拉普拉斯平滑
          1.0
          * (y_x_feature_cnt[x][(y, x_value)][0] + 1) 
          / (class_cnt[y][0] + len(x_feature_cnt[x]))
        )
print y_x_feature_cnt

{'x2': {(-1, 'M'): 0.3333333333333333, (1, 'M'): 0.4166666666666667, (1, 'L'): 0.4166666666666667, (1, 'S'): 0.16666666666666666, (-1, 'S'): 0.4444444444444444, (-1, 'L'): 0.2222222222222222}, 'x1': {(1, 2): 0.3333333333333333, (-1, 1): 0.4444444444444444, (1, 3): 0.4166666666666667, (-1, 3): 0.2222222222222222, (-1, 2): 0.3333333333333333, (1, 1): 0.25}}


In [83]:
# 预测

test_data = {'x1': 2,
        'x2': 'S',
        }

max_property_label = None
max_property_value = -9999999

for y in class_list:
    property_value = math.log(class_cnt[y][0]) #由于先验概率=每个分类的频次/总样本数，而贝叶斯的分类过程，是比较先验概率*独立条件概率的大小，因此先验概率的分母，也可以忽略不计
    for x in feature_list:
        property_value += math.log(y_x_feature_cnt[x][(y, test_data[x])])
    if property_value >= max_property_value:
        max_property_value = property_value
        max_property_label = y

print max_property_label, max_property_value

-1 -0.117783035656
